# Sentiment Labelling and Analysis

In [2]:
# For data manipulation and analysis
import pandas as pd
import numpy as np

# For text preprocessing
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import datetime
import string

# For multilabel classification
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
import os

# For neural networks



# For model evaluation
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

### 1) Reading in the tags dataframe, get the unique set, subset appropriately
- NEEDS TO CONTAIN GLOVE_VEC because moDAL queries based on glove_vec

In [2]:
import pandas as pd

tags_read = pd.read_csv("../dataset/tags_withglovevec.csv")

tags = tags_read

tags['tag'] = tags['tag'].astype('str')

In [2]:
import numpy as np
tags = tags[tags['has_glove_vec'] == True] # needs to contain glove vector


def string_to_vector(s):
    # Remove unwanted characters and split by spaces
    numbers = s.strip('[]').split()
    
    # Convert to float and then to numpy array
    return np.array([float(num) for num in numbers])

tags['glove_vec'] = tags['glove_vec'].apply(string_to_vector)


/var/folders/tk/x3sjpph95kz4ghcssq1py2zc0000gn/T/ipykernel_47722/3865019710.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tags['glove_vec'] = tags['glove_vec'].apply(string_to_vector)


In [3]:
tags

,Unnamed: 0,Unnamed: 0.1,userId,movieId,tag,timestamp,un-lemmatised,glove_vec,has_glove_vec
0,7,266,318,260,s,2015-02-20 22:42:49,NaN,"[0.18209, 0.88297, -0.49805, 0.53137, -0.36084...",True
1,8,267,318,115149,action,2015-02-21 15:58:30,action,"[0.02024, 0.84992, -0.7815, -0.82769, 0.43115,...",True
2,15,274,320,2762,twist,2006-04-25 11:33:52,twist,"[-0.095859, -0.17472, -0.034692, -0.37307, 0.3...",True
3,16,275,320,2959,twist,2006-04-25 11:30:58,twist,"[-0.095859, -0.17472, -0.034692, -0.37307, 0.3...",True
4,17,276,320,3996,overrate,2006-04-25 11:32:28,overrated,"[0.28151, -0.42171, -0.38275, 0.15364, -0.7648...",True
...,...,...,...,...,...,...,...,...,...
50186,109306,390955,138280,116797,history,2015-01-30 23:07:25,history,"[0.045847, 0.074334, 0.015092, -0.26392, 0.155...",True
50187,109307,390956,138280,116797,informatics,2015-01-30 23:07:35,informatics,"[0.17728, 0.15395, 0.77811, 0.16527, 1.2438, 0...",True
50188,109308,390957,138280,116797,mathematics,2015-01-30 23:07:17,mathematics,"[1.0033, 0.38874, 0.64312, -0.6863, 0.93268, 0...",True
50189,109310,390959,138280,117871,image,2015-01-30 23:09:16,image,"[0.011091, 0.48461, 0.019142, 0.083725, 0.5027...",True


In [4]:
print(tags.shape)

(49114, 9)


In [5]:
small_sample = tags.sample(1000)
small_sample.drop_duplicates(subset=['tag'], inplace=True)

In [6]:
tags.drop_duplicates(subset=['tag'], inplace=True) 

/var/folders/tk/x3sjpph95kz4ghcssq1py2zc0000gn/T/ipykernel_47722/2127020140.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tags.drop_duplicates(subset=['tag'], inplace=True)


In [7]:
"Unique tags",  len(tags) 

('Unique tags', 2845)

In [4]:
def generate_tag_subset_report(unique_tags, seed=42):
    # Set random seed to ensure reproducibility
    np.random.seed(seed)
    # Calculate 40% of the total unique tags as the subset size
    subset_size = int(0.40 * unique_tags)
    # Calculate the number of samples for fine-tuning and testing
    fine_tune_size = int(0.75 * subset_size)
    test_size = subset_size - fine_tune_size
    # Print the result summary
    print(f"Subset the dataset - No. of unique tags: {unique_tags}\n")
    print(f"Taking a random subset of 40% of {unique_tags} samples for both fine-tuning and testing we get total: {subset_size} samples to use\n")
    print(f"We use MoDAL to query this 40% first, then\n")
    print(f"    - 75% for fine-tuning = {fine_tune_size} samples")
    print(f"    - 25% for testing = {test_size} samples for testing\n")
    # Create the table data for display
    data = [
        ["No. of unique tags", unique_tags, "100%"],
        ["Subset (40% of Total)", subset_size, "40%"],
        ["Manually labelled", subset_size, "40%"],
        ["Fine-tuning (75% of Subset)", fine_tune_size, "75%"],
        ["Testing (25% of Subset)", test_size, "25%"]
    ]
    # Use pandas to display the table
    df = pd.DataFrame(data, columns=["Subset", "Sample Size", "Percentage"])

    # Print the table
    print(df.to_string(index=False))

# Call the function
generate_tag_subset_report(2845)

Subset the dataset - No. of unique tags: 2845

Taking a random subset of 40% of 2845 samples for both fine-tuning and testing we get total: 1138 samples to use

We use MoDAL to query this 40% first, then

    - 75% for fine-tuning = 853 samples
    - 25% for testing = 285 samples for testing

                     Subset  Sample Size Percentage
         No. of unique tags         2845       100%
      Subset (40% of Total)         1138        40%
          Manually labelled         1138        40%
Fine-tuning (75% of Subset)          853        75%
    Testing (25% of Subset)          285        25%


Labelling the fine-tuning and testing portions -> need to manually label 
- use MoDAL to query
- AutoLabeller to label rest of subset 

**MoDAL**
 Method 2) Active learning: label a few samples, use these labels to suggest other unlabelled examples. 

 - ##### RUN ONCE, THEN DON'T RUN AGAIN because this will produce random initial samples

In [9]:
subset = tags.sample(frac=0.40, random_state=42)
len(subset)


1138

In [12]:
# !pip install git+https://github.com/modAL-python/modAL.git
# !pip show modAL-python


Name: modAL-python
Version: 0.4.2
Summary: A modular active learning framework for Python3
Home-page: https://modAL-python.github.io/
Author: Tivadar Danka
Author-email: 85a5187a@opayq.com
License: MIT
Location: /Users/jiayi/anaconda3/lib/python3.11/site-packages
Requires: numpy, pandas, scikit-learn, scipy, skorch
Required-by: 


In [11]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling

# Initial labeled set
initial_size = 10
labeled_df = subset.sample(n=initial_size)
subset.drop(labeled_df.index, inplace=True)

# Wait for initial labeled data to be annotated manually  ------------- THIS HAS BEEN DONE
print(f"Please manually label the initial set of {initial_size} samples.")
labeled_df.to_csv("../dataset/to_label/initial_samples_to_label.csv")

# # # Pause here until the initial labeled samples are done
# # input(f"Once you've labeled the initial set of samples and saved the file, type 'done' to continue: ")


Please manually label the initial set of 10 samples.


In [22]:
# Read back annotated data
labeled_df = pd.read_csv("../dataset/to_label/initial_samples_to_label_done.csv")
# After reading the annotated data
labeled_df['glove_vec'] = labeled_df['glove_vec'].apply(string_to_vector)


# Data preparation
X_train = np.stack(labeled_df['glove_vec'].to_numpy())
y_train = labeled_df['manual_label'].to_numpy()

# Active learner initialization
learner = ActiveLearner(
    estimator=RandomForestClassifier(),
    query_strategy=uncertainty_sampling,
    X_training=X_train,
    y_training=y_train
)
num_iterations = 10
for i in range(10):
    X_unlabeled = pd.DataFrame(subset['glove_vec'].tolist()).to_numpy()
    X_unlabeled = X_unlabeled[~np.isnan(X_unlabeled).any(axis=1)]
    
    query_idx, query_sample = learner.query(X_unlabeled, n_instances=10)
    
    sample_to_label = subset.iloc[query_idx]
    tags_to_remove = sample_to_label['tag'].tolist()  # Get the list of 'tag' values to remove
    
    sample_to_label.to_csv(f"../dataset/to_label/samples_to_label_{i + 1}.csv")
    
    # Pause for manual labeling
    print(f"Please label iteration {i + 1} samples.")
    
    input("Once you've labeled the samples and saved the file, type 'done' to continue: ")

    annotated_samples = pd.read_csv(f"../dataset/to_label/samples_to_label_{i + 1}_done.csv")
    annotated_samples['glove_vec'] = annotated_samples['glove_vec'].apply(string_to_vector)

    new_labels = annotated_samples['manual_label'].values
    learner.teach(query_sample, new_labels)
    
    # Remove labeled samples from the subset for the next iteration
    subset = subset[~subset['tag'].isin(tags_to_remove)]  # Drop by 'tag'

Please label iteration 7 samples.
Once you've labeled the samples and saved the file, type 'done' to continue: done
Please label iteration 8 samples.
Once you've labeled the samples and saved the file, type 'done' to continue: done
Please label iteration 9 samples.
Once you've labeled the samples and saved the file, type 'done' to continue: done
Please label iteration 10 samples.
Once you've labeled the samples and saved the file, type 'done' to continue: sonw


Combining the labelled samples into one dataframe

In [23]:

# Define the number of files you have (for this example, let's say you have 5 files)
num_files = 10

# Create an empty list to store individual dataframes
dfs = []

# Loop through each file, read its content, and append it to the dfs list
for i in range(num_files):
    file_path = f"../dataset/to_label/samples_to_label_{i + 1}_done.csv"
    df = pd.read_csv(file_path)
    dfs.append(df)

# Concatenate all dataframes in the dfs list
annotated_samples = pd.concat(dfs, ignore_index=True)


In [24]:
annotated_samples

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,userId,movieId,tag,timestamp,un-lemmatised,glove_vec,has_glove_vec,manual_label,Unnamed: 11
0,5036,10069,26471,7434,98809,exaggerated,2013/5/21 21:02,exaggerated,[ 0.40403 -0.59134 0.063463 -0.12254 ...,True,negative,NaN
1,16671,35520,96184,27898,89030,trivial,2014/11/2 20:16,trivial,[ 0.35444 0.70451 0.21284 -0.47087 ...,True,negative,NaN
2,3123,5700,19104,4550,2353,nonsensical,2014/7/15 07:52,nonsensical,[ 0.70851 0.036539 0.40354 0.035616 0.35...,True,negative,NaN
3,33810,74148,232951,82747,6666,meaningless,2013/4/16 09:09,meaningless,[ 4.5875e-01 5.4953e-01 -1.4093e-01 1.1588e-...,True,negative,NaN
4,396,617,10068,2062,63072,pointless,2010/9/21 20:21,pointless,[-0.045273 0.20127 -0.060488 0.16788 ...,True,negative,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,366,566,10017,2062,1981,plot,2010/10/5 00:12,plot,[-3.0523e-01 -6.0794e-01 -4.5547e-02 -2.5703e-...,True,neutral,NaN
96,18750,40027,126696,42290,50,sensational,2006/2/8 21:06,sensational,[-5.7019e-01 3.3709e-01 -8.3342e-01 -7.3405e-...,True,negative,NaN
97,1517,2852,15409,4087,112183,pretentious,2015/2/6 23:19,pretentious,[ 7.6172e-01 -1.9364e-01 -2.0250e-01 -1.5498e-...,True,neutral,NaN
98,9110,19474,59884,16724,54272,simpson,2008/6/18 01:06,simpsons,[-4.0774e-01 3.2613e-01 -6.3297e-01 -1.9871e-...,True,neutral,NaN


In [25]:
annotated_samples.drop(columns=['Unnamed: 0.2','Unnamed: 0.1', 'Unnamed: 0', 'userId', 'movieId', 'timestamp', 'un-lemmatised', 'glove_vec', 'has_glove_vec'], inplace=True)

In [26]:
annotated_samples.to_csv('../dataset/annotated_samples_comb.csv')

In [27]:
rest_of_subset_label = subset.loc[~subset['tag'].isin(annotated_samples['tag'])]
# this is the rest of the rows that need to be labelled


rest_of_subset_label.to_csv("../dataset/rest_of_subset_label.csv") # this is what will be labelled in the autolabelleer part

### Label the rest of the dataset - manually

In [12]:

# reading in the file
rest_of_subset_labelled = pd.read_csv("../dataset/rest_of_subset_label_done.csv")
rest_of_subset_labelled = rest_of_subset_labelled.rename(columns={'label' : 'manual_label'})

In [14]:
# join to annotated_samples
annotated_samples = pd.read_csv("../dataset/annotated_samples_comb.csv")

df_labelled = pd.concat([rest_of_subset_labelled, annotated_samples], ignore_index=True)


In [33]:
df_labelled

,Unnamed: 0,userId,movieId,tag,timestamp,un-lemmatised,glove_vec,has_glove_vec,manual_label,Unnamed: 11
0,1339.0,4087.0,91128.0,caribbean,2012/11/3 20:42,caribbean,[ 0.010196 0.096712 0.011479 -0.72511 ...,True,neutral,NaN
1,25963.0,58612.0,41997.0,athens,2007/3/18 13:52,athens,[ 3.5467e-01 1.6093e-01 1.2579e-01 -2.0789e-...,True,neutral,NaN
2,6549.0,9815.0,5878.0,compassionate,2013/12/12 22:22,compassionate,[-0.69978 -0.031807 -0.14433 0.40873 ...,True,positive,NaN
3,5383.0,8771.0,380.0,buddy,2006/1/21 05:59,buddy,[ 4.7541e-01 -2.7164e-01 8.2356e-02 -1.0867e-...,True,positive,NaN
4,8213.0,14947.0,1968.0,clumsy,2010/11/2 22:27,clumsy,[ 0.47254 -0.29096 -0.27617 -0.26583 -...,True,neutral,NaN
...,...,...,...,...,...,...,...,...,...,...
1113,NaN,NaN,NaN,plot,NaN,NaN,NaN,NaN,neutral,NaN
1114,NaN,NaN,NaN,sensational,NaN,NaN,NaN,NaN,negative,NaN
1115,NaN,NaN,NaN,pretentious,NaN,NaN,NaN,NaN,neutral,NaN
1116,NaN,NaN,NaN,simpson,NaN,NaN,NaN,NaN,neutral,NaN


### Sentiment Model - cardiffnlp/twitter-roberta-base-sentiment-latest using the "fine_tune" set

- Positive, Negative, Neutral


Only run the below if you need to rerun the sentiment again (takes approx 211minutes)
- Assigns a sentiment label, number to each tag from the tags dataset

In [3]:
# !pip install torch torchvision torchaudio
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # 如果安装了 CUDA，可以检查它是否可用


2.4.0
False


In [5]:
# code ref: https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
# MODEL = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

config = AutoConfig.from_pretrained(MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)


def roberta_sentiment(tag):
    enc_inp = tokenizer(tag, return_tensors='pt', padding=True, truncation=True)
    sentiment = model(**enc_inp)

    scores = sentiment.logits.detach().numpy()[0]
    scores = softmax(scores)

    

    # label and scores:
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    top_label = config.id2label[ranking[0]]
    top_score = np.round(float(scores[ranking[0]]), 4)
    
    return (top_label, top_score)


tags['sentiment_roberta'] = tags['tag'].apply(roberta_sentiment)


/Users/jiayi/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


/Users/jiayi/anaconda3/lib/python3.11/site-packages/transformers/modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_

Output file of tag, sentiment:

In [6]:
tags[["tag", "sentiment_roberta"]].to_csv("../dataset/robertaTest.csv")

### Scaling the Sentiment Scores 

Raw sentiment score labels with ranges:

- Neutral (0,1)
- Positive (0,1)
- Negative (0,1)

Need to change these so that all are on a 0,1 scale

Where:

- Neutral = 0.5
- Positive = 1
- Negative = -1

In [7]:
# loading back in the sentiment results
sentiment_roberta = pd.read_csv("../dataset/robertaTest.csv")

In [8]:
df_s = sentiment_roberta
df_s[['sentiment_label', 'sentiment_value']] = df_s['sentiment_roberta'].str.extract(r"\('(.+)', ([\d.]+)\)")

# Convert sentiment_value to float
df_s['sentiment_value'] = df_s['sentiment_value'].astype(float)

# Initialize a new column for scaled sentiment values
df_s['scaled_sentiment_value'] = np.nan

# For 'neutral' labels, scale the value so that stronger neutral scores are closer to 0
df_s.loc[df_s['sentiment_label'] == 'neutral', 'scaled_sentiment_value'] = (1 - df_s['sentiment_value']) * 0.5

# For 'positive' labels, scale the value so that stronger positive scores are closer to 1
df_s.loc[df_s['sentiment_label'] == 'positive', 'scaled_sentiment_value'] = (df_s['sentiment_value'] * 0.5) + 0.5

# For 'negative' labels, scale the value to [-1 if sentiment_value is 1, otherwise scale to (-value)]
df_s.loc[(df_s['sentiment_label'] == 'negative') & (df_s['sentiment_value'] == 1), 'scaled_sentiment_value'] = -1
df_s.loc[(df_s['sentiment_label'] == 'negative') & (df_s['sentiment_value'] < 1), 'scaled_sentiment_value'] = -df_s['sentiment_value']

# Display the DataFrame
df_s[df_s['sentiment_label'] == 'positive'].head(100)

,Unnamed: 0,tag,sentiment_roberta,sentiment_label,sentiment_value,scaled_sentiment_value
10,10,hero,"('positive', 0.6957)",positive,0.6957,0.84785
60,60,humorous,"('positive', 0.5959)",positive,0.5959,0.79795
64,64,classic,"('positive', 0.5635)",positive,0.5635,0.78175
68,68,classic,"('positive', 0.5635)",positive,0.5635,0.78175
93,93,nature,"('positive', 0.5058)",positive,0.5058,0.75290
...,...,...,...,...,...,...
1855,1855,classic,"('positive', 0.5635)",positive,0.5635,0.78175
1856,1856,classic,"('positive', 0.5635)",positive,0.5635,0.78175
1863,1863,christmas,"('positive', 0.4902)",positive,0.4902,0.74510
1879,1879,inspirational,"('positive', 0.6262)",positive,0.6262,0.81310


Output the df_s to a file that is read in by the CB model and also the sentimentMod.ipynb for clustering and CF prediction

In [9]:
df_s.to_csv('../dataset/df_tag_sentiment.csv',index=False)

In [10]:
df_s.head(10)

,Unnamed: 0,tag,sentiment_roberta,sentiment_label,sentiment_value,scaled_sentiment_value
0,0,s,"('neutral', 0.5123)",neutral,0.5123,0.24385
1,1,action,"('neutral', 0.6874)",neutral,0.6874,0.15630
2,2,twist,"('neutral', 0.7575)",neutral,0.7575,0.12125
3,3,twist,"('neutral', 0.7575)",neutral,0.7575,0.12125
4,4,overrate,"('neutral', 0.5669)",neutral,0.5669,0.21655
5,5,violent,"('neutral', 0.485)",neutral,0.4850,0.25750
6,6,violent,"('neutral', 0.485)",neutral,0.4850,0.25750
7,7,end,"('neutral', 0.5769)",neutral,0.5769,0.21155
8,8,music,"('neutral', 0.4884)",neutral,0.4884,0.25580
9,9,fight,"('neutral', 0.6423)",neutral,0.6423,0.17885


In [15]:

df_labelled = df_labelled.rename(columns={'manual_label' : 'sentiment_label'})


In [16]:
### Evaluating sentiment model on manually labelled dataset
df_s[["tag", "sentiment_label"]].drop_duplicates()


,tag,sentiment_label
0,s,neutral
1,action,neutral
2,twist,neutral
4,overrate,neutral
5,violent,neutral
...,...,...
50062,newspaper,neutral
50110,static,positive
50139,repeat,neutral
50142,seal,neutral


In [17]:
df_labelled

,Unnamed: 0,userId,movieId,tag,timestamp,un-lemmatised,glove_vec,has_glove_vec,sentiment_label,Unnamed: 11
0,1339,4087.0,91128.0,caribbean,2012/11/3 20:42,caribbean,[ 0.010196 0.096712 0.011479 -0.72511 ...,True,neutral,NaN
1,25963,58612.0,41997.0,athens,2007/3/18 13:52,athens,[ 3.5467e-01 1.6093e-01 1.2579e-01 -2.0789e-...,True,neutral,NaN
2,6549,9815.0,5878.0,compassionate,2013/12/12 22:22,compassionate,[-0.69978 -0.031807 -0.14433 0.40873 ...,True,positive,NaN
3,5383,8771.0,380.0,buddy,2006/1/21 05:59,buddy,[ 4.7541e-01 -2.7164e-01 8.2356e-02 -1.0867e-...,True,positive,NaN
4,8213,14947.0,1968.0,clumsy,2010/11/2 22:27,clumsy,[ 0.47254 -0.29096 -0.27617 -0.26583 -...,True,neutral,NaN
...,...,...,...,...,...,...,...,...,...,...
1113,95,NaN,NaN,plot,NaN,NaN,NaN,NaN,neutral,NaN
1114,96,NaN,NaN,sensational,NaN,NaN,NaN,NaN,negative,NaN
1115,97,NaN,NaN,pretentious,NaN,NaN,NaN,NaN,neutral,NaN
1116,98,NaN,NaN,simpson,NaN,NaN,NaN,NaN,neutral,NaN


In [18]:
merged_df = pd.merge(df_labelled, df_s, on='tag', how='inner', suffixes=('_test', '_df_s'))

# Then, calculate the accuracy by comparing 'sentiment_label' from both dataframes
correct_predictions = (merged_df['sentiment_label_test'] == merged_df['sentiment_label_df_s']).sum()
total_predictions = len(merged_df)

accuracy = correct_predictions / total_predictions if total_predictions != 0 else 0

print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 81.32%
